In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import scipy.io as sci

def data_prepare(X,Y,N,traj_length):               # regularize trajectories for training of the inference values
    import numpy as np 
    thr=1e-10                            
    x = np.array(X).reshape(N,traj_length)             # reshape
    x = np.diff(x,axis=1)                              # take the increments 
    sx = np.std(x,axis=1)                 
    x = (x-np.mean(x,axis=1).reshape(len(x),1)) / np.where(sx>thr,sx,1).reshape(len(x),1)   # normalize data
    
    label_inf=np.zeros((N,4))
    label_inf[:,0]=Y[:,3]                              # the first exponent
    label_inf[:,1]=Y[:,5]                              # the second exponent
    label_inf[:,2]=np.sin((2*np.pi*Y[:,1])/200)        # sine of the switching time
    label_inf[:,3]=np.cos((2*np.pi*Y[:,1])/200)        # cosine of the switching time
    
    label_c1 = []
    label_c1.append(np.equal(Y[:,2],0))                # if the first model is attm
    label_c1.append(np.equal(Y[:,2],1))                # if the first model is ctrw
    label_c1.append(np.equal(Y[:,2],2))                # if the first model is sbm
    label_c1.append(np.equal(Y[:,2],3))                # if the first model is lw
    label_c1.append(np.equal(Y[:,2],4))                # if the first model is fbm
    label_c1 = np.array(np.transpose(label)) + 0                            

    label_c2 = []
    label_c2.append(np.equal(Y[:,2],0))                # if the second model is attm
    label_c2.append(np.equal(Y[:,2],1))                # if the second model is ctrw
    label_c2.append(np.equal(Y[:,2],2))                # if the second model is sbm
    label_c2.append(np.equal(Y[:,2],3))                # if the second model is lw
    label_c2.append(np.equal(Y[:,2],4))                # if the second model is fbm
    label_c2 = np.array(np.transpose(label)) + 0                            
    
    return(r, label_inf, label_c1, label_c2)

In [ ]:
###### Building the recurrent neural networks #####

block_size = 4                                             # Size of the blocks of data points

###### Building the recurrent neural network for inference #####

model_segmentation_inf = Sequential()

model_segmentation_inf.add(LSTM(250,                       # first layer: LSTM of dimension 250
                         return_sequences=True,            # return sequences for the second LSTM layer            
                         recurrent_dropout=0.2,            # recurrent dropout for preventing overtraining
                         input_shape=(None, block_size)))  # input shape
                                                           
model_segmentation_inf.add(LSTM(50,                          # second layer: LSTM of dimension 50
                        dropout=0,
                        recurrent_dropout=0.2))

model_segmentation_inf.add(Dense(20))                        # dense layer 

model_segmentation_inf.add(Dense(4))                         # output layer                             

model_segmentation_inf.compile(optimizer='adam',
                               loss='mse', 
                               metrics=['mae'])

model_segmentation_inf.summary()                             # Printing a summary of the built network


###### Building the recurrent neural network for classifying the first model #####

model_segmentation_c1 = Sequential()

model_segmentation_c1.add(LSTM(250,                       # first layer: LSTM of dimension 250
                         return_sequences=True,            # return sequences for the second LSTM layer            
                         recurrent_dropout=0.2,            # recurrent dropout for preventing overtraining
                         input_shape=(None, block_size)))  # input shape
                                                           
model_segmentation_c1.add(LSTM(50,                          # second layer: LSTM of dimension 50
                        dropout=0,
                        recurrent_dropout=0.2))

model_segmentation_c1.add(Dense(20))                        # dense layer 

model_segmentation_c1.add(Dense(5,                         # output layer
                                activation="softmax",))

model_segmentation_c1.compile(optimizer='adam',
                              loss="categorical_crossentropy",
                              metrics=["categorical_accuracy"])

model_segmentation_c1.summary()                             # Printing a summary of the built network



###### Building the recurrent neural network for classifying the second model #####

model_segmentation_c2 = Sequential()

model_segmentation_c2.add(LSTM(250,                       # first layer: LSTM of dimension 250
                         return_sequences=True,            # return sequences for the second LSTM layer            
                         recurrent_dropout=0.2,            # recurrent dropout for preventing overtraining
                         input_shape=(None, block_size)))  # input shape
                                                           
model_segmentation_c2.add(LSTM(50,                          # second layer: LSTM of dimension 50
                        dropout=0,
                        recurrent_dropout=0.2))

model_segmentation_c2.add(Dense(20))                        # dense layer 

model_segmentation_c2.add(Dense(5,                          # output layer
                                activation="softmax",))

model_segmentation_c2.compile(optimizer='adam',
                              loss="categorical_crossentropy",
                              metrics=["categorical_accuracy"])

model_segmentation_c2.summary()                             # Printing a summary of the built network

In [ ]:
###### Training the recurrent neural networks #####

dimension = 1                # 1, 2 or 3 Dimensions
N = 100000                   # Number of trajectories per datasets
traj_length = 225            # Length of the trajectories

batch_sizes = [32, 128, 512, 2048]
dataset_used = [1, 4, 5, 20]
number_epochs = [5, 4, 3, 2]
n = 0

for batch in range(len(batch_sizes)):    
    for repeat in range(dataset_used[batch]):
        data = sci.loadmat(r'data\segmentation\ ' + str(dimension) + 'D_' + str(traj_length) + '_' + str(n) + '.mat')
        n += 1
        X = data['X'][0][0]
        Y = data['Y'][0][0].reshape(N,)
        
        x, label_inf, label_c1, label_c2 = data_prepare(X,Y,N,traj_length)
        
        model_segmentation_inf.fit(x.reshape(N,int(traj_length/block_size),block_size),
                                 label_inf, 
                                 epochs=number_epochs[batch], 
                                 batch_size=batch_sizes[batch],
                                 validation_split=0.1,
                                 shuffle=True)
        
        model_segmentation_c1.fit(x.reshape(N,int(traj_length/block_size),block_size),
                                 label_c1, 
                                 epochs=number_epochs[batch], 
                                 batch_size=batch_sizes[batch],
                                 validation_split=0.1,
                                 shuffle=True)
        
        model_segmentation_c2.fit(x.reshape(N,int(traj_length/block_size),block_size),
                                 label_c2, 
                                 epochs=number_epochs[batch], 
                                 batch_size=batch_sizes[batch],
                                 validation_split=0.1,
                                 shuffle=True)